Dataset: UCI ML Drug Review dataset (for link: https://www.kaggle.com/jessicali9530/kuc-hackathon-winter-2018) 

Train set consists of 161297 samples, test set consists of 53766 samples of drug reviews.
There are several use cases of this dataset: 
- Classification: Can you predict the patient's condition based on the review?
- Regression: Can you predict the rating of the drug based on the review?
- Sentiment analysis: What elements of a review make it more helpful to others? Which patients tend to have more negative reviews? Can you determine if a review is positive, neutral, or negative?

In this notebook, we'll try to classify the text based on the "condition" label. 

In [ ]:
# import packages
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style; style.use('ggplot')
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#read data
train_data = pd.read_csv("../input/kuc-hackathon-winter-2018/drugsComTrain_raw.csv", sep=',')

train_data.head()


In [ ]:
train_data['review'][0]

In [ ]:
#read data
test_data = pd.read_csv("../input/kuc-hackathon-winter-2018/drugsComTest_raw.csv", sep=',')

test_data.head()


In [ ]:
#check if features of train&test datasets are same or not
list(train_data) == list(test_data)

In [ ]:
#check the sizes of train&test datasets(train-test-split: 75:25)
train_data.values.shape[0], test_data.values.shape[0], (train_data.values.shape[0] / test_data.values.shape[0])

In [ ]:
#unique values of "condition" col
train_data.loc[:,"condition"].unique().size, test_data.loc[:, "condition"].unique().size

In [ ]:
#unique values of "drugName" col
train_data.loc[:, "drugName"].unique().size, test_data.loc[:, "drugName"].unique().size

In [ ]:
conditions = train_data["condition"].value_counts().sort_values(ascending=False)
conditions[:10]

# Data Visualization

In [ ]:
top_10_drugs=train_data["drugName"].value_counts().sort_values(ascending=False)[:10]

top_10_drugs_df=pd.DataFrame(zip(top_10_drugs.index, top_10_drugs), columns = ["drugName", "count"])


In [ ]:
import seaborn as sns 

#counts of top 10 drugs
sns.set(style="whitegrid")
plt.figure(figsize=(10, 5))
ax = sns.barplot(x="drugName", y="count", data=top_10_drugs_df, palette=sns.color_palette("cubehelix", 10))
plt.xticks(rotation=90)
plt.title("Top-10 most used drugs", {"fontname":"fantasy", "fontweight":"bold", "fontsize":"medium"})
plt.ylabel("count", {"fontname": "serif", "fontweight":"bold"})
plt.xlabel("drug name", {"fontname": "serif", "fontweight":"bold"})

In [ ]:
#ratings of top-10 drugs 
rating_top_10_df=train_data.loc[train_data["drugName"].isin(top_10_drugs.index), :]
rating_top_10_df

In [ ]:
#ratings(average-variance) of top-10 drugs
sns.set(style="whitegrid")
plt.figure(figsize=(10, 5))
ax = sns.barplot(x="drugName", y="rating", data=rating_top_10_df, palette=sns.color_palette("RdBu_r", 10))
plt.xticks(rotation=90)
plt.title("Average ratings of drugs", {"fontname":"fantasy", "fontweight":"bold", "fontsize":"medium"})
plt.ylabel("average rating&variance", {"fontname": "serif", "fontweight":"bold"})
plt.xlabel("drug name", {"fontname": "serif", "fontweight":"bold"})

In [ ]:
top_10_condition=train_data["condition"].value_counts().sort_values(ascending=False)[:10]

top_10_condition_df=pd.DataFrame(zip(top_10_condition.index, top_10_condition), columns = ["condition", "count"])


In [ ]:
top_10_condition_df

In [ ]:
#counts of top-10 conditions

sns.set(style="whitegrid")
plt.figure(figsize=(10, 5))
ax = sns.barplot(x="condition", y="count", data=top_10_condition_df, palette=sns.cubehelix_palette(10, start=.5, rot=-.20))
plt.xticks(rotation=90)
plt.title("Top-10 conditions", {"fontname":"fantasy", "fontweight":"bold", "fontsize":"medium"})
plt.ylabel("count", {"fontname": "serif", "fontweight":"bold"})
plt.xlabel("condition", {"fontname": "serif", "fontweight":"bold"})

In [ ]:
#count of ratings, people rates mostly if very bad or very good

sns.set(style="whitegrid")
plt.figure(figsize=(10, 5))
ax = sns.countplot(x="rating", data=train_data, palette=sns.cubehelix_palette(10, start=.5, rot=-.20))
plt.title("Rating counts", {"fontname":"fantasy", "fontweight":"bold", "fontsize":"medium"})
plt.ylabel("count", {"fontname": "serif", "fontweight":"bold"})
plt.xlabel("rating", {"fontname": "serif", "fontweight":"bold"})

In [ ]:
#conditions of top-1000 review(according to the "usefulCount" values)

top_100_reviews=train_data["usefulCount"].sort_values(ascending=False)[:100]
top_100_reviews_df=train_data.loc[top_100_reviews.index, :]


In [ ]:
print(top_100_reviews_df["usefulCount"].max())
print(top_100_reviews_df["usefulCount"].min())

In [ ]:
#conditions of most useful(popular) reviews 

sns.set(style="whitegrid")
plt.figure(figsize=(10, 5))
ax = sns.countplot(x="condition", data=top_100_reviews_df, palette=sns.color_palette("RdBu_r", 10))
plt.xticks(rotation=90)
plt.title("Conditions of top-100 reviews", {"fontname":"fantasy", "fontweight":"bold", "fontsize":"medium"})
plt.ylabel("count", {"fontname": "serif", "fontweight":"bold"})
plt.xlabel("condition", {"fontname": "serif", "fontweight":"bold"})

In [ ]:
#create a new feature review_length
train_data["review_length"]=train_data["review"].str.len()

test_data["review_length"]=test_data["review"].str.len()

In [ ]:
#density of the review length
plt.figure(figsize=(12.8,6))
sns.distplot(train_data['review_length']).set_title('review length distribution')

In [ ]:
quantile_95 = train_data['review_length'].quantile(0.95)
df_95 = train_data[train_data['review_length'] < quantile_95]

plt.figure(figsize=(12.8,6))
sns.distplot(df_95['review_length']).set_title('review length distribution')

In [ ]:
quantile_95

In [ ]:
df_more10k = train_data[train_data['review_length'] > 10000]
print(len(df_more10k))
print("-------")
print(df_more10k["review"].values)

In [ ]:
#box-plot

top_10=train_data["condition"].value_counts().sort_values(ascending=False)[:10]
top_10_df=train_data.loc[train_data["condition"].isin(top_10.index), :]

plt.figure(figsize=(16,6))
sns.boxplot(data=top_10_df, x='condition', y='review_length', width=.5)
plt.title("Box-plot", {"fontname":"fantasy", "fontweight":"bold", "fontsize":"medium"})
plt.ylabel("review length", {"fontname": "serif", "fontweight":"bold"})
plt.xlabel("condition", {"fontname": "serif", "fontweight":"bold"})

In [ ]:
#box-plot

top_10_version2=df_95["condition"].value_counts().sort_values(ascending=False)[:10]
top_10_version2_df=df_95.loc[df_95["condition"].isin(top_10.index), :]

plt.figure(figsize=(16,6))
sns.boxplot(data=top_10_version2_df, x='condition', y='review_length', width=.5)
plt.title("Box-plot of 0.95 quantile", {"fontname":"fantasy", "fontweight":"bold", "fontsize":"medium"})
plt.ylabel("review length", {"fontname": "serif", "fontweight":"bold"})
plt.xlabel("condition", {"fontname": "serif", "fontweight":"bold"})

length distribution of conditions are not very different from each other, so if we choose top-10 conditions as our labels, we can say that they are almost normalized. 

Length of the dataset will reduce to 73k from 160k, if we choose top-10 conditions.

In [ ]:
top_10_condition=train_data["condition"].value_counts().sort_values(ascending=False)[:10]
top_10_condition_df=train_data.loc[train_data["condition"].isin(top_10_condition.index), :]
len(top_10_condition_df)

In [ ]:
import pickle

with open('top_10_train.pickle', 'wb') as output:
    pickle.dump(top_10_condition_df, output)

In [ ]:
#pickle for test_data(selecting top-10 conditions)
top_10_condition_test=test_data["condition"].value_counts().sort_values(ascending=False)[:10]
top_10_condition_test_df=test_data.loc[test_data["condition"].isin(top_10_condition_test.index), :]
len(top_10_condition_test_df)


In [ ]:
import pickle
with open('top_10_test.pickle', 'wb') as output:
    pickle.dump(top_10_condition_test_df, output)